# Colab: Parity Swap Pipeline (2-slot end-of-day)

This notebook is strict and deterministic.

Expected mirror layout (exact paths):
- `/content/drive/MyDrive/wunderfund_mirror/valid.parquet`
- `/content/drive/MyDrive/wunderfund_mirror/vanilla_all/gru_parity_v1_seed42.pt` ... `seed64.pt`

No Drive-wide searching is used.

In [ ]:
from google.colab import drive
from pathlib import Path
import os, shutil

drive.mount('/content/drive')

REPO_URL = 'https://github.com/vincentvdo6/competition_package.git'
BRANCH = 'master'
REPO = Path('/content/competition_package')

MIRROR = Path('/content/drive/MyDrive/wunderfund_mirror')
MIRROR_VALID = MIRROR / 'valid.parquet'
MIRROR_CKPT_DIR = MIRROR / 'vanilla_all'

assert MIRROR_VALID.exists(), f'Missing valid parquet: {MIRROR_VALID}'
for s in range(42, 65):
    p = MIRROR_CKPT_DIR / f'gru_parity_v1_seed{s}.pt'
    assert p.exists(), f'Missing checkpoint: {p}'

print('Mirror checks passed')

In [ ]:
import subprocess

if REPO.exists():
    shutil.rmtree(REPO)

subprocess.run(['git', 'clone', '--branch', BRANCH, REPO_URL, str(REPO)], check=True)
os.chdir(REPO)

subprocess.run(['git', 'pull', 'origin', BRANCH], check=True)
assert (REPO / 'scripts' / 'run_colab_parity_swap_pipeline.py').exists()
assert (REPO / 'scripts' / 'discover_parity_swaps.py').exists()
print('Repo ready')

In [ ]:
import shutil

(REPO / 'datasets').mkdir(parents=True, exist_ok=True)
(REPO / 'logs').mkdir(parents=True, exist_ok=True)

dst_ckpt_dir = REPO / 'logs' / 'vanilla_all'
if dst_ckpt_dir.exists():
    shutil.rmtree(dst_ckpt_dir)
dst_ckpt_dir.mkdir(parents=True, exist_ok=True)

for s in range(42, 65):
    src = MIRROR_CKPT_DIR / f'gru_parity_v1_seed{s}.pt'
    dst = dst_ckpt_dir / src.name
    shutil.copy2(src, dst)

dst_valid = REPO / 'datasets' / 'valid.parquet'
if dst_valid.exists() or dst_valid.is_symlink():
    dst_valid.unlink()
shutil.copy2(MIRROR_VALID, dst_valid)

print('Copied checkpoints:', len(list(dst_ckpt_dir.glob('gru_parity_v1_seed*.pt'))))
print('Valid parquet exists:', dst_valid.exists())

In [ ]:
import subprocess, sys

cmd = [
    sys.executable, 'scripts/run_colab_parity_swap_pipeline.py',
    '--repo', '.',
    '--cache-dir', 'cache/parity23_valid_preds',
    '--data', 'datasets/valid.parquet',
    '--output-report', 'logs/parity_swap_discovery_feb21_b2.json',
    '--output-dir', 'submissions/ready',
    '--slot-a-name', 'feb21-b2-t1swap-a-onnx.zip',
    '--slot-b-name', 'feb21-b2-t0swap-a-onnx.zip',
    '--bootstrap', '200',
    '--delta-min', '0.00015',
    '--p10-max-drop', '0.00005',
]

proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
for line in proc.stdout:
    print(line, end='', flush=True)
rc = proc.wait()
print('exit_code:', rc)
assert rc == 0, 'Pipeline failed; scroll logs above.'

In [ ]:
import subprocess, sys, glob

for z in sorted(glob.glob('submissions/ready/feb21-b2-*.zip')):
    print('\n===', z, '===')
    subprocess.run([sys.executable, 'scripts/check_submission_zip.py', z], check=True)

print('\nArtifacts:')
print('report: logs/parity_swap_discovery_feb21_b2.json')
print('zip A : submissions/ready/feb21-b2-t1swap-a-onnx.zip')
print('zip B : submissions/ready/feb21-b2-t0swap-a-onnx.zip')